### These pieces of code are to recover contaminated probe intensities 

#### author: Yanan Qin

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
import os.path
import shutil
import random
import nibabel as nib
import numpy as np
import glob
import sys
import cv2
import tensorflow as tf
from keras import backend as K
from keras.backend.tensorflow_backend import set_session
from keras.models import load_model
from tensorflow import Graph
from os import path
import matplotlib.pyplot as plt

/home/yananq/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/yananq/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/yananq/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/yananq/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dty

In [175]:
# get the name of all 1810 positive cases
pos_case = set()
for i in os.listdir('/media/ssd2/yananq/mace/code/pos_pred'):
    pos_case.add('.'.join(i.split('.')[:-2]))
print(len(pos_case)) 

#get the name of all 37724 samples
all_sample = set(os.listdir('/media/ssd2/yananq/mace/data/raw_image/mace_full_png'))

1810


In [197]:
#contam_dir.txt save the dir of all 1810 contaminated cel file
# for R code use
count=0
temp=set()
path_file=open('/media/ssd/yananq/mace/contam_dir.txt','w') 
for series in os.listdir('/media/ssd/yananq/mace/bg'):
    for sample in os.listdir('/media/ssd/yananq/mace/bg/'+series):
        if sample in pos_case and sample not in temp:
            temp.add(sample)
            count+=1
            path_file.write('/media/ssd/yananq/mace/bg/'+series+'/'+sample+'\n')
        
#/media/ssd/yananq/mace/bg does not contain all the samples; find other samples from enlarge dir 
for i in pos_case:
    if i not in temp:
        if i in os.listdir('/media/ssd/yananq/mace/enlarged/post_vel'):
            temp.add(i)
            count+=1
            path_file.write('/media/ssd/yananq/mace/enlarged/post_vel/'+i+'\n')
        elif i in os.listdir('/media/ssd/yananq/mace/enlarged/prior_vel'):
            temp.add(i)
            count+=1
            path_file.write('/media/ssd/yananq/mace/enlarged/prior_vel/'+i+'\n')
count        

1704

In [190]:
# pos_recover_dir.txt saves the dir of all negative cases cel files out of all 37724 samples
# used for R code to calculate reference matrix
pos_dir = set()
count = 0
for series in os.listdir('/media/ssd/yananq/mace/bg'):
    for sample in os.listdir('/media/ssd/yananq/mace/bg/'+series):
        if sample not in pos_case and (sample+'.png') in all_sample:
            pos_dir.add('/media/ssd/yananq/mace/bg/'+series+'/'+sample)
print(len(pos_dir))  #number of total negative samples
path_file=open('/media/ssd/yananq/mace/pos_recover_dir.txt','w') 
for i in pos_dir:
    if i.lower().endswith('cel') and i.lower().startswith('/media/ssd/yananq/mace/bg'):
        path_file.write(i+'\n')
        count +=1
count

35178


35178

### Call R code recover.R

### Recover (code in recover_intensity.py and recover_save_to_map_txt.py)

###  Overall, % of the studies are affected by this correction

In [3]:
#set of all recovered samples
recovered = set(os.listdir('/media/ssd/yananq/mace/recovered_contam_intensities'))
len(recovered)    

1703

In [4]:
# check each study as well as the samples under the study. If the sample name exist in the recovered set, 
# save the name of the study
count = 0
affected_study = set()
for i in os.listdir('/media/ssd/yananq/mace/bg'):
    for j in os.listdir('/media/ssd/yananq/mace/bg/'+i):
        if j+'.txt' in recovered:
            affected_study.add(i)

In [5]:
len(affected_study)/3165 #3165 is the number of studies in total

0.26666666666666666